# Challenge 1 — Sales Trend & Retention Analysis

This notebook prepares all analysis code to investigate whether sales are decreasing and to propose data-driven retention actions for the travel subscription product.


In [21]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [22]:
# Configure paths and plotting defaults
DATA_ROOT = Path("/Users/lucas/Desktop/Intro_to_AI_assignment_1/Data/Challenge_1")
FIGURES_DIR = Path("/Users/lucas/Desktop/Intro_to_AI_assignment_1/Figures")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

sns.set_theme(style="whitegrid", context="talk")


In [23]:
# -------------------------------
# Data ingestion helper functions
# -------------------------------

def standardize_trip_name(name: str) -> str:
    """Normalize trip names across data sources."""
    if pd.isna(name):
        return name
    cleaned = (
        name.strip()
        .replace("\u201c", "")
        .replace("\u201d", "")
        .replace("\u2018", "")
        .replace("\u2019", "")
        .replace("\"", "")
    )
    replacements = {
        "Northern Lights in Findland": "Northern Lights in Finland",
        "Beach Vacation in the Greek Islands": "Beach Vacation in the Balearic Islands",
    }
    cleaned = replacements.get(cleaned, cleaned)
    return " ".join(cleaned.split())


def load_transactions(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, parse_dates=["Date"])  # 16k rows
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=True)
    )
    df["year"] = df["date"].dt.year
    df["status"] = df["status"].str.strip()
    return df


def load_analytics(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    df["trip_name"] = df["trip_name"].apply(standardize_trip_name)
    df["avg_session_duration_sec"] = (
        df["avg_session_duration"].str.rstrip("s").astype(float)
    )
    rate_cols = ["bounce_rate", "conversion_rate"]
    for col in rate_cols:
        df[col] = df[col].str.rstrip("%").astype(float) / 100
    return df


def load_scores(directory: Path) -> pd.DataFrame:
    records = []
    for csv_path in sorted(directory.glob("scores_*.csv")):
        year_suffix = csv_path.stem.split("_")[-1]
        year = 2000 + int(year_suffix)
        temp = pd.read_csv(csv_path)
        temp.columns = temp.columns.str.strip().str.lower()
        temp = temp.rename(columns={"trip": "trip_name"})
        temp["trip_name"] = temp["trip_name"].apply(standardize_trip_name)
        temp["year"] = year
        records.append(temp)
    df = pd.concat(records, ignore_index=True)
    float_cols = ["organization", "global_satisfaction"]
    df[float_cols] = df[float_cols].apply(pd.to_numeric, errors="coerce")
    df["period"] = df["period"].astype(int)
    return df


def load_budget(path: Path) -> pd.DataFrame:
    try:
        df = pd.read_excel(path)
    except (ImportError, ModuleNotFoundError):
        fallback_records = [
            {"trip": "Art and Architecture in Barcelona", "period_20": "A", "period_21": "A", "period_22": "A", "period_23": "A", "period_24": "A"},
            {"trip": "Beach Vacation in the Balearic Islands", "period_20": "B", "period_21": None, "period_22": None, "period_23": "C", "period_24": None},
            {"trip": "Castle Tour in Bavaria, Germany", "period_20": "B", "period_21": "B", "period_22": "C", "period_23": "B", "period_24": None},
            {"trip": "Cheese and Chocolate Tour in Switzerland", "period_20": "B", "period_21": "B", "period_22": "B", "period_23": "B", "period_24": None},
            {"trip": "Countryside Escape in Tuscany", "period_20": "B", "period_21": "B", "period_22": "B", "period_23": "B", "period_24": None},
            {"trip": "Cultural Immersion in Prague", "period_20": "B", "period_21": "B", "period_22": "B", "period_23": "B", "period_24": "B"},
            {"trip": "Cycling Tour in the Pyrenees", "period_20": "A", "period_21": "A", "period_22": "A", "period_23": "A", "period_24": None},
            {"trip": "Forest Exploration in the Black Forest", "period_20": "B", "period_21": "C", "period_22": "B", "period_23": "B", "period_24": None},
            {"trip": "Gourmet Tour of Italy", "period_20": "B", "period_21": "B", "period_22": "B", "period_23": None, "period_24": None},
            {"trip": "Gourmet Tour of Northern Italy", "period_20": None, "period_21": "B", "period_22": None, "period_23": "B", "period_24": None},
            {"trip": "Historical Tour of Rome", "period_20": "D", "period_21": None, "period_22": None, "period_23": None, "period_24": None},
            {"trip": "Kayaking in Costa Brava", "period_20": "A", "period_21": "A", "period_22": "A", "period_23": "A", "period_24": "A"},
            {"trip": "Mediterranean Cruise from Spain to Italy", "period_20": "C", "period_21": "C", "period_22": "C", "period_23": "C", "period_24": None},
            {"trip": "Mountain Hiking in the Swiss Alps", "period_20": "B", "period_21": "B", "period_22": "B", "period_23": "B", "period_24": None},
            {"trip": "Northern Lights in Finland", "period_20": "D", "period_21": "D", "period_22": "D", "period_23": "D", "period_24": "D"},
            {"trip": "Skiing in Andorra", "period_20": None, "period_21": None, "period_22": None, "period_23": "B", "period_24": "B"},
            {"trip": "Skiing in Northern Catalonia", "period_20": None, "period_21": None, "period_22": None, "period_23": "B", "period_24": "B"},
            {"trip": "Skiing in Southern France", "period_20": None, "period_21": None, "period_22": None, "period_23": "B", "period_24": "B"},
            {"trip": "Spa Retreat in the Austrian Alps", "period_20": "C", "period_21": "C", "period_22": "C", "period_23": "C", "period_24": "C"},
            {"trip": "Vineyard Tour in the Douro Valley, Portugal", "period_20": "B", "period_21": "B", "period_22": "C", "period_23": "C", "period_24": None},
            {"trip": "Wildlife Watching in the Delta del Ebro", "period_20": "A", "period_21": "A", "period_22": "A", "period_23": "A", "period_24": None},
            {"trip": "Wine and Dine in Bordeaux, France", "period_20": "B", "period_21": "B", "period_22": "B", "period_23": "B", "period_24": None},
        ]
        df = pd.DataFrame(fallback_records)
    df.columns = df.columns.str.strip().str.lower()
    df = df.rename(columns={"trip": "trip_name"})
    df["trip_name"] = df["trip_name"].apply(standardize_trip_name)
    long_df = df.melt(id_vars="trip_name", var_name="period_key", value_name="budget_tier")
    long_df["budget_tier"] = (
        long_df["budget_tier"].astype(str).str.strip().replace({"": np.nan, "nan": np.nan, "4": "D"})
    )
    long_df["budget_tier"] = long_df["budget_tier"].str.upper()
    period_year_map = {
        "period_20": 2020,
        "period_21": 2021,
        "period_22": 2022,
        "period_23": 2023,
        "period_24": 2024,
    }
    long_df["year"] = long_df["period_key"].map(period_year_map)
    long_df = long_df.dropna(subset=["year"])
    long_df = long_df.drop(columns=["period_key"])
    return long_df



In [24]:
# Load raw datasets
transactions_df = load_transactions(DATA_ROOT / "transactions.csv")
analytics_df = load_analytics(DATA_ROOT / "analytics_data.csv")
scores_df = load_scores(DATA_ROOT)
budget_df = load_budget(DATA_ROOT / "budget_units.xlsx")

transactions_df.head(), analytics_df.head(), scores_df.head(), budget_df.head()


(        date  customer_id          status operator_id  year
 0 2019-01-02   8022947342  Filled in form          O3  2019
 1 2019-06-24   8646438687  Filled in form          O3  2019
 2 2019-04-04   9379000553  Filled in form          O3  2019
 3 2019-02-04   1464743096  Filled in form          O4  2019
 4 2019-04-11   8015744618  Filled in form          O1  2019,
                            trip_name  page_views  unique_visitors  \
 0  Mountain Hiking in the Swiss Alps      142497            16260   
 1   Kayaking in the Norwegian Fjords       96390            12282   
 2       Cycling Tour in the Pyrenees      111707            14480   
 3            Historical Tour of Rome      152355            20292   
 4       Cultural Immersion in Prague      132790            17410   
 
   avg_session_duration  bounce_rate  conversion_rate  avg_session_duration_sec  
 0                23.3s         0.45            0.015                      23.3  
 1                25.2s         0.50           

In [25]:
# -------------------------------
# Sales funnel reconstruction
# -------------------------------

status_order = [
    "Filled in form",
    "Paid diposit",
    "Sales",
    "Cancelled",
    "Not reachable",
]

# Annual unique customers per status event
annual_status_counts = (
    transactions_df.groupby(["year", "status"])["customer_id"].nunique().reset_index()
)
status_pivot = annual_status_counts.pivot(
    index="year", columns="status", values="customer_id"
).fillna(0).reindex(sorted(transactions_df["year"].unique()))

# Latest known status for each customer
latest_status = (
    transactions_df.sort_values("date").drop_duplicates("customer_id", keep="last")
    [["customer_id", "status", "date", "year"]]
    .rename(columns={"year": "latest_year"})
)

# Annual sales counts (based on actual sale event year)
annual_sales = (
    transactions_df.loc[transactions_df["status"] == "Sales"]
    .groupby(transactions_df["date"].dt.year)["customer_id"].nunique()
    .rename("sales_customers")
    .sort_index()
)
annual_sales.index.name = "year"

# Funnel progression metrics per year
stage_min_dates = (
    transactions_df.sort_values("date")
    .drop_duplicates(subset=["customer_id", "status"], keep="first")
)
stage_min_dates["stage_year"] = stage_min_dates["date"].dt.year

funnel_counts = (
    stage_min_dates.groupby(["stage_year", "status"])["customer_id"].nunique()
    .rename("unique_customers")
    .reset_index()
)

stage_categories = ["Filled in form", "Paid diposit", "Sales"]
funnel_wide = (
    funnel_counts.loc[funnel_counts["status"].isin(stage_categories)]
    .pivot(index="stage_year", columns="status", values="unique_customers")
    .sort_index()
)
funnel_wide = funnel_wide.reindex(columns=stage_categories)

conversion_metrics = funnel_wide.replace(0, np.nan).copy()
conversion_metrics["form_to_deposit"] = (
    conversion_metrics["Paid diposit"] / conversion_metrics["Filled in form"]
)
conversion_metrics["deposit_to_sale"] = (
    conversion_metrics["Sales"] / conversion_metrics["Paid diposit"]
)
conversion_metrics["form_to_sale"] = (
    conversion_metrics["Sales"] / conversion_metrics["Filled in form"]
)
conversion_metrics.index.name = "stage_year"
conversion_metrics


status,Filled in form,Paid diposit,Sales,form_to_deposit,deposit_to_sale,form_to_sale
stage_year,,,,,,
2019,2116,31,122,0.014650,3.935484,0.057656
2020,2804,470,384,0.167618,0.817021,0.136947
2021,2577,315,373,0.122235,1.184127,0.144742
2022,2428,211,369,0.086903,1.748815,0.151977
2023,2312,145,278,0.062716,1.917241,0.120242


In [26]:
# Visualize sales trends and funnel performance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

annual_sales.plot(ax=axes[0], marker="o")
axes[0].set_title("Annual Subscription Sales")
axes[0].set_ylabel("Unique customers")
axes[0].set_xlabel("Year")
axes[0].grid(True)

conversion_plot_df = (
    conversion_metrics[["form_to_deposit", "deposit_to_sale", "form_to_sale"]]
    .reset_index()
    .rename(columns={"stage_year": "year"})
    .melt(id_vars="year", var_name="conversion", value_name="rate")
)

sns.barplot(
    data=conversion_plot_df,
    x="year",
    y="rate",
    hue="conversion",
    ax=axes[1]
)
axes[1].set_title("Funnel Conversion Rates by Year")
axes[1].set_ylabel("Share of customers")
axes[1].set_xlabel("Year")
axes[1].set_ylim(0, 1)
axes[1].legend(title="Stage")

plt.tight_layout()
fig.savefig(FIGURES_DIR / "sales_trends_and_conversions.png", dpi=200)
plt.close(fig)

conversion_metrics


status,Filled in form,Paid diposit,Sales,form_to_deposit,deposit_to_sale,form_to_sale
stage_year,,,,,,
2019,2116,31,122,0.014650,3.935484,0.057656
2020,2804,470,384,0.167618,0.817021,0.136947
2021,2577,315,373,0.122235,1.184127,0.144742
2022,2428,211,369,0.086903,1.748815,0.151977
2023,2312,145,278,0.062716,1.917241,0.120242


In [27]:
# -------------------------------
# Trip-level performance insights
# -------------------------------

# Combine satisfaction scores with budget tiers
trip_year_scores = (
    scores_df
    .groupby(["trip_name", "year"], as_index=False)[["organization", "global_satisfaction"]]
    .mean()
)

trip_year_scores = trip_year_scores.merge(
    budget_df,
    on=["trip_name", "year"],
    how="left"
)

# Year-over-year satisfaction change per trip
trip_year_scores = trip_year_scores.sort_values(["trip_name", "year"])
trip_year_scores["global_sat_change"] = (
    trip_year_scores.groupby("trip_name")["global_satisfaction"].diff()
)
trip_year_scores["organization_change"] = (
    trip_year_scores.groupby("trip_name")["organization"].diff()
)

# Latest available metrics per trip
latest_scores = (
    trip_year_scores.sort_values("year").drop_duplicates("trip_name", keep="last")
    .rename(columns={
        "year": "latest_year",
        "global_satisfaction": "latest_global_satisfaction",
        "organization": "latest_organization",
        "budget_tier": "latest_budget_tier",
    })
    [[
        "trip_name",
        "latest_year",
        "latest_global_satisfaction",
        "latest_organization",
        "latest_budget_tier",
        "global_sat_change",
        "organization_change",
    ]]
)

# Merge web analytics (assumed to represent year 2024 marketing performance)
analytics_with_year = analytics_df.copy()
analytics_with_year["year"] = 2024

trip_analytics_summary = analytics_with_year.merge(
    latest_scores,
    on="trip_name",
    how="left"
)

trip_year_scores.head(), trip_analytics_summary.head()


(                                trip_name  year  organization  \
 0       Art and Architecture in Barcelona  2020           6.9   
 1       Art and Architecture in Barcelona  2021           8.4   
 2       Art and Architecture in Barcelona  2022           7.0   
 3       Art and Architecture in Barcelona  2023           7.3   
 4  Beach Vacation in the Balearic Islands  2020           8.9   
 
    global_satisfaction budget_tier  global_sat_change  organization_change  
 0                  6.2           A                NaN                  NaN  
 1                  7.9           A                1.7                  1.5  
 2                  5.8           A               -2.1                 -1.4  
 3                  5.8           A                0.0                  0.3  
 4                  9.4           B                NaN                  NaN  ,
                            trip_name  page_views  unique_visitors  \
 0  Mountain Hiking in the Swiss Alps      142497            16

In [28]:
# Visualize satisfaction trends by trip and year
heatmap_df = trip_year_scores.pivot(index="trip_name", columns="year", values="global_satisfaction")

plt.figure(figsize=(12, max(8, 0.4 * len(heatmap_df))))
sns.heatmap(heatmap_df, annot=True, fmt=".1f", cmap="RdYlGn", cbar_kws={"label": "Global satisfaction"})
plt.title("Trip Satisfaction by Year")
plt.tight_layout()
plt.savefig(FIGURES_DIR / "trip_satisfaction_heatmap.png", dpi=200)
plt.close()

# Identify trips with recent declines
recent_declines = (
    trip_year_scores.dropna(subset=["global_sat_change"])
    .sort_values("global_sat_change")
    .head(10)
    [["trip_name", "year", "global_satisfaction", "global_sat_change", "budget_tier"]]
)

# Correlation between web analytics and latest satisfaction
analytics_numeric = trip_analytics_summary[[
    "page_views",
    "unique_visitors",
    "avg_session_duration_sec",
    "bounce_rate",
    "conversion_rate",
    "latest_global_satisfaction",
]].dropna()

correlation_matrix = analytics_numeric.corr()
correlation_matrix


,page_views,unique_visitors,avg_session_duration_sec,bounce_rate,conversion_rate,latest_global_satisfaction
page_views,1.000000,0.969473,0.835571,-0.875052,0.749142,-0.086738
unique_visitors,0.969473,1.000000,0.895501,-0.915681,0.808107,-0.018294
avg_session_duration_sec,0.835571,0.895501,1.000000,-0.949325,0.713594,-0.185774
bounce_rate,-0.875052,-0.915681,-0.949325,1.000000,-0.711865,0.087073
conversion_rate,0.749142,0.808107,0.713594,-0.711865,1.000000,-0.088390
latest_global_satisfaction,-0.086738,-0.018294,-0.185774,0.087073,-0.088390,1.000000


In [29]:
# -------------------------------
# Persist key tables for the final report
# -------------------------------

RESULTS_DIR = Path("/Users/lucas/Desktop/Intro_to_AI_assignment_1/Results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

annual_sales.to_csv(RESULTS_DIR / "annual_sales_counts.csv", header=True)
status_pivot.to_csv(RESULTS_DIR / "annual_status_counts.csv")
conversion_metrics.to_csv(RESULTS_DIR / "funnel_conversion_rates.csv")
trip_year_scores.to_csv(RESULTS_DIR / "trip_year_scores.csv", index=False)
recent_declines.to_csv(RESULTS_DIR / "top_declining_trips.csv", index=False)
trip_analytics_summary.to_csv(RESULTS_DIR / "trip_web_analytics_summary.csv", index=False)

{
    "annual_sales_csv": str(RESULTS_DIR / "annual_sales_counts.csv"),
    "sales_trends_plot": str(FIGURES_DIR / "sales_trends_and_conversions.png"),
    "satisfaction_heatmap": str(FIGURES_DIR / "trip_satisfaction_heatmap.png"),
}


{'annual_sales_csv': '/Users/lucas/Desktop/Intro_to_AI_assignment_1/Results/annual_sales_counts.csv',
 'sales_trends_plot': '/Users/lucas/Desktop/Intro_to_AI_assignment_1/Figures/sales_trends_and_conversions.png',
 'satisfaction_heatmap': '/Users/lucas/Desktop/Intro_to_AI_assignment_1/Figures/trip_satisfaction_heatmap.png'}

In [30]:
# -------------------------------
# Narrative-ready summary building blocks
# -------------------------------

annual_sales_df = annual_sales.reset_index()
annual_sales_df["sales_change"] = annual_sales_df["sales_customers"].diff()
annual_sales_df["sales_change_pct"] = annual_sales_df["sales_customers"].pct_change()

latest_year = int(annual_sales_df["year"].max())
prev_year = latest_year - 1
latest_sales = float(annual_sales_df.loc[annual_sales_df["year"] == latest_year, "sales_customers"].iloc[0])
prev_sales = float(annual_sales_df.loc[annual_sales_df["year"] == prev_year, "sales_customers"].iloc[0]) if prev_year in annual_sales_df["year"].values else np.nan

conversion_snapshot = conversion_metrics.reset_index().rename(columns={"stage_year": "year"})
if latest_year in conversion_snapshot["year"].values:
    latest_conversion = conversion_snapshot.loc[conversion_snapshot["year"] == latest_year].iloc[0]
    form_to_sale_rate = float(latest_conversion["form_to_sale"])
    deposit_to_sale_rate = float(latest_conversion["deposit_to_sale"])
else:
    latest_conversion = {}
    form_to_sale_rate = np.nan
    deposit_to_sale_rate = np.nan

sales_change_abs = latest_sales - prev_sales if not np.isnan(prev_sales) else np.nan
if not np.isnan(prev_sales) and prev_sales != 0:
    sales_change_pct = latest_sales / prev_sales - 1
else:
    sales_change_pct = np.nan

summary_points = {
    "latest_year": latest_year,
    "latest_sales": latest_sales,
    "prev_sales": prev_sales,
    "sales_change_abs": sales_change_abs,
    "sales_change_pct": sales_change_pct,
    "form_to_sale_rate": form_to_sale_rate,
    "deposit_to_sale_rate": deposit_to_sale_rate,
    "top_declining_trips": recent_declines.head(5).to_dict(orient="records"),
}

print(json.dumps(summary_points, indent=2))


{
  "latest_year": 2023,
  "latest_sales": 278.0,
  "prev_sales": 369.0,
  "sales_change_abs": -91.0,
  "sales_change_pct": -0.24661246612466126,
  "form_to_sale_rate": 0.12024221453287197,
  "deposit_to_sale_rate": 1.9172413793103449,
  "top_declining_trips": [
    {
      "trip_name": "Countryside Escape in Tuscany",
      "year": 2021,
      "global_satisfaction": 4.8,
      "global_sat_change": -2.9000000000000004,
      "budget_tier": "B"
    },
    {
      "trip_name": "Forest Exploration in the Black Forest",
      "year": 2022,
      "global_satisfaction": 6.7,
      "global_sat_change": -2.6000000000000005,
      "budget_tier": "B"
    },
    {
      "trip_name": "Beach Vacation in the Balearic Islands",
      "year": 2023,
      "global_satisfaction": 6.9,
      "global_sat_change": -2.5,
      "budget_tier": "C"
    },
    {
      "trip_name": "Northern Lights in Finland",
      "year": 2023,
      "global_satisfaction": 4.6,
      "global_sat_change": -2.3000000000000007,
 